In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [26]:
from fastai.vision import ImageList, PointsItemList, get_transforms, ResizeMethod
import json
import pandas as pd
import os
import torch
import torch.nn as nn
from torch.nn.functional import mse_loss
import re
from shutil import copyfile

In [3]:
images_path = '../data/images'
labels_path = '../data/landmarks.csv'
target_size = 640

In [4]:
def load_coordinates_to_dataframe(labels_path):
    df = pd.read_csv(labels_path)
    
    df.columns.values[-1] = "filename"
    df['filename'] = df['filename'].str.replace('images/', '')
#     df = df.set_index('filename')
    
    df = df.drop(['box/_top', 'box/_left', 'box/_width', 'box/_height'], axis=1)
    for col in df:
        if (col[-5:] == '_name'):
            df = df.drop([col], axis=1) 

    df.columns = df.columns.str.replace('box/part/', '')
    df.columns = df.columns.str.replace('/', '')

    return df

In [5]:
labels = load_coordinates_to_dataframe('./../data/landmarks.csv')

In [24]:
labels = labels.set_index('filename')

In [25]:
labels.loc['scottish_terrier_119.jpg']

# TODO get rid of images with no coords

KeyError: 'scottish_terrier_119.jpg'

In [6]:
labels.head()

,0_x,0_y,1_x,1_y,2_x,2_y,3_x,3_y,4_x,4_y,...,6_y,7_x,7_y,8_x,8_y,9_x,9_y,10_x,10_y,filename
0,339,131,339,78,380,101,419,77,417,131,...,122,392,123,366,149,373,144,385,149,Abyssinian_1.jpg
1,156,198,189,102,245,125,330,79,324,180,...,192,293,185,253,254,274,245,292,250,Abyssinian_100.jpg
2,95,235,28,80,196,108,317,33,318,191,...,210,251,201,180,308,212,294,243,299,Abyssinian_102.jpg
3,268,165,243,101,302,103,354,71,361,143,...,156,337,148,317,193,326,190,338,188,Abyssinian_103.jpg
4,251,171,239,106,305,106,371,106,357,168,...,182,328,182,293,224,304,223,315,225,Abyssinian_105.jpg


In [23]:
labels[labels['filename'] == 'scottish_terrier_119.jpg']

,0_x,0_y,1_x,1_y,2_x,2_y,3_x,3_y,4_x,4_y,...,6_y,7_x,7_y,8_x,8_y,9_x,9_y,10_x,10_y,filename


In [19]:
def get_y_func(x):
    filename = str(x).split('/')[-1]
    print(filename)
    coords = labels[labels['filename'] == filename].iloc[0][:-1]
    return torch.tensor(coords, dtype=torch.float)

In [20]:
transforms = get_transforms(do_flip=False, max_zoom=1.1, max_warp=0.01, max_rotate=45) 

In [21]:
data = (
    ImageList
#         .from_df(df=labels, path='.', cols='filename', folder=images_path)
        .from_folder(images_path)
        .split_by_rand_pct(valid_pct=0.15)
        .label_from_func(get_y_func, label_cls=PointsItemList)
#         .transform(
#             transforms, 
#             size=160, 
#             tfm_y=True, 
#             remove_out=False,  
#             padding_mode='border', 
#             resize_method=ResizeMethod.PAD
#         )
        .databunch(bs=32) 
#         .normalize(imagenet_stats)
)

Egyptian_Mau_144.jpg
scottish_terrier_119.jpg


IndexError: single positional indexer is out-of-bounds

In [22]:
data.x[0]

NameError: name 'data' is not defined

In [25]:
data.y[0]

AttributeError: 'Tensor' object has no attribute 'read'

In [26]:
data.show_batch()

AttributeError: Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/PIL/Image.py", line 2613, in open
    fp.seek(0)
AttributeError: 'Tensor' object has no attribute 'seek'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/torch/utils/data/_utils/worker.py", line 99, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/opt/anaconda3/lib/python3.7/site-packages/torch/utils/data/_utils/worker.py", line 99, in <listcomp>
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/opt/anaconda3/lib/python3.7/site-packages/fastai/data_block.py", line 648, in __getitem__
    if self.item is None: x,y = self.x[idxs],self.y[idxs]
  File "/opt/anaconda3/lib/python3.7/site-packages/fastai/data_block.py", line 118, in __getitem__
    if isinstance(idxs, Integral): return self.get(idxs)
  File "/opt/anaconda3/lib/py

In [28]:
data.show_batch(3, figsize=(9,6))

AttributeError: Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/PIL/Image.py", line 2613, in open
    fp.seek(0)
AttributeError: 'Tensor' object has no attribute 'seek'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/torch/utils/data/_utils/worker.py", line 99, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/opt/anaconda3/lib/python3.7/site-packages/torch/utils/data/_utils/worker.py", line 99, in <listcomp>
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/opt/anaconda3/lib/python3.7/site-packages/fastai/data_block.py", line 648, in __getitem__
    if self.item is None: x,y = self.x[idxs],self.y[idxs]
  File "/opt/anaconda3/lib/python3.7/site-packages/fastai/data_block.py", line 118, in __getitem__
    if isinstance(idxs, Integral): return self.get(idxs)
  File "/opt/anaconda3/lib/py